In [25]:
# imports
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

In [2]:
# model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("./FinBertModel", repo_type="model", local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained("./FinBertModel", local_files_only=True)


In [4]:
# loading in the data
df = catalog.load('processed_article_data')
df.head()

[10/23/23 08:15:38] INFO     Loading data from 'processed_article_data' (CSVDataSet)...         ]8;id=33996;file:///usr/local/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=897618;file:///usr/local/lib/python3.11/site-packages/kedro/io/data_catalog.py#492\492]8;;\

,date,time,headline,full_text,author,type_of_author
0,2023-10-13,19:32:01,Safe-haven gold rallies over 3% as Middle East...,Oct 13 (Reuters) - Gold prices jumped more tha...,Ashitha Shivaprasad,Person
1,2023-10-12,04:00:07,"Gold subdued as yields, dollar rise after US i...",By Ashitha Shivaprasad 2 Min Read (Reuters) - ...,Ashitha Shivaprasad,Person
2,2023-10-12,10:27:31,PRECIOUS-Gold scales two-week peak with focus ...,By Harshit Verma 3 Min Read Gold hits highest ...,Harshit Verma,Person
3,2023-10-12,04:01:01,PRECIOUS-Gold climbs as markets await US CPI d...,By Swati Verma 2 Min Read Gold hits highest si...,Swati Verma,Person
4,2023-10-12,01:40:10,PRECIOUS-Gold firm ahead of US CPI data on cau...,By Reuters Staff 2 Min Read Oct 12 (Reuters) -...,Reuters Staff,Person


In [50]:
sample_text = df['full_text'].head(40)
sample_text = sample_text.tolist()
sample_text


[
    'Oct 13 (Reuters) - Gold prices jumped more than 3% on Friday and were poised for their best week in seven months as the intensifying conflict in the Middle East sent investors scurrying for safe-haven assets. Zero-yield bullion got an additional fillip from expectations that the U.S. interest rates may have peaked. Spot gold was up 3.2% at $1,928.15 per ounce by 0309 p.m. ET (1908 GMT). U.S. gold futures settled 3.1% higher at $1,941.50. Prices were up 5.2% for the week. Investors kept a tab on developments in the Middle East conflict, which has unnerved markets since the start of the week. Israel said its infantry and tanks had carried out raids inside the Gaza Strip, its first announcement of a shift from an air war to ground operations to root out Hamas fighters a week after their deadly rampage in southern Israel. This fuelled inflows into assets considered to be safe havens such as gold. "Investors are fleeing to safe havens as the risks of Middle East tensions grow," said

In [51]:
# removing 'by' lines
for text in sample_text:
    sub_str = "Reporting by"
    re = text.split(sub_str)
    sub_str2 = "(Reuters) - "
    re = re[0].split(sub_str2)
    print(re[1])

Gold prices jumped more than 3% on Friday and were poised for their best week in seven months as the intensifying conflict in the Middle East sent investors scurrying for safe-haven assets. Zero-yield bullion got an additional fillip from expectations that the U.S. interest rates may have peaked. Spot gold was up 3.2% at $1,928.15 per ounce by 0309 p.m. ET (1908 GMT). U.S. gold futures settled 3.1% higher at $1,941.50. Prices were up 5.2% for the week. Investors kept a tab on developments in the Middle East conflict, which has unnerved markets since the start of the week. Israel said its infantry and tanks had carried out raids inside the Gaza Strip, its first announcement of a shift from an air war to ground operations to root out Hamas fighters a week after their deadly rampage in southern Israel. This fuelled inflows into assets considered to be safe havens such as gold. "Investors are fleeing to safe havens as the risks of Middle East tensions grow," said Edward Moya, senior market

In [53]:
# need to summarize text to less 512 characters

In [52]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
results = classifier(sample_text)

for result in results:
    print(result)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)                │
│ ❱ 2 results = classifier(sample_text)                                                            │
│   3                                                                                              │
│   4 for result in results:                                                                       │
│   5 │   print(result)                                                                            │
│                                                                                                  │
│ /home/vscode/.local/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:1 │
│ 56 in __call__                                                                                   │
│                                                                                                  │
│   153 │   │   │                                                                                  │
│   154 │   │   │   If `top_k` is used, one such dictionary is returned per label.                 │
│   155 │   │   """                                                                                │
│ ❱ 156 │   │   result = super().__call__(*args, **kwargs)                                         │
│   157 │   │   # TODO try and retrieve it in a nicer way from _sanitize_parameters.               │
│   158 │   │   _legacy = "top_k" not in kwargs                                                    │
│   159 │   │   if isinstance(args[0], str) and _legacy:                                           │
│                                                                                                  │
│ /home/vscode/.local/lib/python3.11/site-packages/transformers/pipelines/base.py:1121 in __call__ │
│                                                                                                  │
│   1118 │   │   │   │   final_iterator = self.get_iterator(                                       │
│   1119 │   │   │   │   │   inputs, num_workers, batch_size, preprocess_params, forward_params,   │
│   1120 │   │   │   │   )                                                                         │
│ ❱ 1121 │   │   │   │   outputs = list(final_iterator)                                            │
│   1122 │   │   │   │   return outputs                                                            │
│   1123 │   │   │   else:                                                                         │
│   1124 │   │   │   │   return self.run_multi(inputs, preprocess_params, forward_params, postpro  │
│                                                                                                  │
│ /home/vscode/.local/lib/python3.11/site-packages/transformers/pipelines/pt_utils.py:124 in       │
│ __next__                                                                                         │
│                                                                                                  │
│   121 │   │   │   return self.loader_batch_item()                                                │
│   122 │   │                                                                                      │
│   123 │   │   # We're out of items within a batch                                                │
│ ❱ 124 │   │   item = next(self.iterator)                                                         │
│   125 │   │   processed = self.infer(item, **self.params)                                        │
│   126 │   │   # We now have a batch of "inferred things".                                        │
│   127 │   │   if self.loader_batch_size is not None:       